### TPC Benchmark - Query History  
Note: NB_06, NB_07 and NB_08 use shared variables. Run in order 6, 7, 8.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 1.0})

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import config, sfa, bqa, tools

In [2]:
%store -r results_dir

In [3]:
results_dir

'/home/colin/code/bq_snowflake_benchmark/results/results_bqsf_ds_100GB_01_test-01_2020-07-10_09:32:27.385122'

In [4]:
def exp_log_t0(results_dir):
    """Get query history for experiment"""
    fps_time = glob.glob(results_dir + config.sep + "benchmark_times*")
    source_csv = {k:v for k,v in zip([os.path.basename(x).split("_")[2] for x in fps_time], fps_time)}
    dfsf = pd.read_csv(source_csv["sf"])
    dfbq = pd.read_csv(source_csv["bq"])
    df = pd.concat([dfsf, dfbq])
    t0 = df.driver_t0.min()
    t0 = pd.to_datetime(t0)
    return t0

In [5]:
t0 = exp_log_t0(results_dir)
t0

Timestamp('2020-07-10 16:32:28.756704+0000', tz='UTC')

In [6]:
t_buffer = pd.Timedelta("20 minutes")
t_buffer

Timedelta('0 days 00:20:00')

In [7]:
sf = sfa.SFTPC(test="ds",
               scale=1,
               cid="01",
               warehouse="TEST9000",
               desc="query_history",
               verbose=False,
               verbose_query=False)
sf.connect()
df_sf_history_sq, qid_sf = sf.query_history(t0=pd.to_datetime(t0) - t_buffer,
                                            t1=pd.Timestamp.now())

sf.close()

In [8]:
df_sf_history_sq.shape

(0, 41)

In [9]:
%store df_sf_history_sq

Stored 'df_sf_history_sq' (DataFrame)


In [10]:
sf = sfa.AU(warehouse="TEST9000")
sf.connect()

df_sf_history_av, qid_sf = sf.query_history_view(t0=pd.to_datetime(t0) - t_buffer,
                                                 t1=pd.Timestamp.utcnow())

sf.close()

In [11]:
df_sf_history_av.shape

(207, 59)

In [12]:
%store df_sf_history_av

Stored 'df_sf_history_av' (DataFrame)


In [13]:
df_sf_history_av.to_csv(results_dir + config.sep + "query_history_sf.csv")

In [14]:
bq = bqa.BQTPC(test="ds",
               scale=1,
               cid="01",
               desc="query_history",
               verbose=False,
               verbose_query=True)

df_bq_history, qid_bq = bq.query_history(t0=pd.to_datetime(t0) - t_buffer,
                                         t1=pd.Timestamp.utcnow())

BIGQUERY QUERY TEXT
select * from `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT where job_type = 'QUERY' and end_time between '2020-07-10 16:12:28' AND '2020-07-10 17:39:39'



In [15]:
%store df_bq_history

Stored 'df_bq_history' (DataFrame)


In [16]:
df_bq_history.shape

(103, 21)

In [17]:
df_bq_history

,creation_time,project_id,project_number,user_email,job_id,job_type,statement_type,start_time,end_time,query,state,reservation_id,total_bytes_processed,total_slot_ms,error_result,cache_hit,destination_table,referenced_tables,labels,timeline,job_stages
0,2020-07-10 16:47:40.032000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,cee20f95-ecff-4b05-9592-a4590d541101,QUERY,SELECT,2020-07-10 16:47:40.189000+00:00,2020-07-10 16:47:40.690000+00:00,\nselect distinct(i_product_name)\n from item...,DONE,None,14945614,384,None,False,"{'project_id': 'tpc-benchmarking-9432', 'datas...","[{'project_id': 'tpc-benchmarking-9432', 'data...","[{'key': 'label', 'value': 'ds_100gb_01-q41-1-...","[{'elapsed_ms': 477, 'total_slot_ms': 384, 'pe...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1..."
1,2020-07-10 16:46:44.046000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,57714a80-e57d-4938-a021-f4ebd697f176,QUERY,SELECT,2020-07-10 16:46:44.198000+00:00,2020-07-10 16:46:44.849000+00:00,\nselect \n\tcount(*) as r1\nfrom store_sales\...,DONE,None,6602940020,63047,None,False,"{'project_id': 'tpc-benchmarking-9432', 'datas...","[{'project_id': 'tpc-benchmarking-9432', 'data...","[{'key': 'label', 'value': 'ds_100gb_01-q96-1-...","[{'elapsed_ms': 622, 'total_slot_ms': 63047, '...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1..."
2,2020-07-10 16:52:17.280000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,9135d0c4-71bf-40a0-b1fc-a9618a96ab79,QUERY,SELECT,2020-07-10 16:52:17.499000+00:00,2020-07-10 16:52:18.276000+00:00,"\nselect i_brand_id brand_id, i_brand brand,\...",DONE,None,6714733437,75195,None,False,"{'project_id': 'tpc-benchmarking-9432', 'datas...","[{'project_id': 'tpc-benchmarking-9432', 'data...","[{'key': 'label', 'value': 'ds_100gb_01-q55-1-...","[{'elapsed_ms': 620, 'total_slot_ms': 75096, '...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1..."
3,2020-07-10 16:51:18.464000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,85988636-1945-47fc-8af0-8a5f65a68cb4,QUERY,SELECT,2020-07-10 16:51:18.605000+00:00,2020-07-10 16:51:19.340000+00:00,"\nselect s_store_name, s_store_id,\n s...",DONE,None,6602557221,77108,None,False,"{'project_id': 'tpc-benchmarking-9432', 'datas...","[{'project_id': 'tpc-benchmarking-9432', 'data...","[{'key': 'label', 'value': 'ds_100gb_01-q43-1-...","[{'elapsed_ms': 617, 'total_slot_ms': 76783, '...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1..."
4,2020-07-10 16:47:08.711000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,19025228-ebbc-4b91-907f-b31bfa43faee,QUERY,SELECT,2020-07-10 16:47:08.864000+00:00,2020-07-10 16:47:09.692000+00:00,"\nselect dt.d_year \n ,item.i_brand_id ...",DONE,None,6714733453,74555,None,False,"{'project_id': 'tpc-benchmarking-9432', 'datas...","[{'project_id': 'tpc-benchmarking-9432', 'data...","[{'key': 'label', 'value': 'ds_100gb_01-q3-1-t...","[{'elapsed_ms': 628, 'total_slot_ms': 73626, '...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1..."
5,2020-07-10 17:11:26.959000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,89ed0709-cddb-4532-93cf-b3f10ffd1e6d,QUERY,SELECT,2020-07-10 17:11:27.106000+00:00,2020-07-10 17:11:27.935000+00:00,"\nselect dt.d_year\n \t,item.i_category_id\n ...",DONE,None,6712645644,64358,None,False,"{'project_id': 'tpc-benchmarking-9432', 'datas...","[{'project_id': 'tpc-benchmarking-9432', 'data...","[{'key': 'label', 'value': 'ds_100gb_01-q42-1-...","[{'elapsed_ms': 613, 'total_slot_ms': 63570, '...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1..."
6,2020-07-10 16:48:23.369000+00:00,tpc-benchmarking-9432,476262002685,tpc-service-account-2@tpc-benchmarking-9432.ia...,7709c3c9-6523-4f77-969e-f515dac85794,QUERY,SELECT,2020-07-10 16:48:23.539000+00:00,2020-07-10 16:48:24.250000+00:00,"\nselect dt.d_year\n 

In [18]:
df_bq_history.to_csv(results_dir + config.sep + "query_history_bq.csv")